In [ ]:
### permitted modules ###
import numpy as np
import datetime
import hashlib
#########################

### comments about code and algorithms used (optional) ###

##########################################################

In [ ]:
# Utils
def custom_base_to_decimal(number, base):
	digits = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
	
	negative = False

	if number[0] == '-':
		negative = True
		number = number[1::]

	
	decimal_value = 0

	for i, digit in enumerate(reversed(number)):
		decimal_value += digits.index(digit) * (base ** i)

	if negative:
		return decimal_value * -1
	return decimal_value

def binary_to_dec(binary):
	return int(binary, 2)

def dec_to_binary(decimal):
	binary_str = bin(decimal)
	return binary_str[2:].zfill(14)

def convert_to_custom_base(number, base):
	result = ""

	while number > 0:
		remainder = int(number % base)
		#print("rem", remainder)
		result = chr(remainder) + result
		number //= base
	return result

def number_to_base(number, base):
	if not isinstance(number, int) or not isinstance(base, int) or base < 2 or base > 32:
		raise ValueError("Invalid input. 'number' and 'base' must be integers, and 'base' should be between 2 and 36.")

	digits = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
	
	negative = False

	if number < 0:
		negative = True

	number = abs(number)
	result = ""

	if number == 0:
		return "0"

	while number > 0:
		remainder = number % base
		result = digits[remainder] + result
		number //= base

	#print("base", result)
	if negative:
		return "-" + result
	return result

In [ ]:
###  utility constants, functions and lookups can be instantiated/built here ###

def expand_line(line, line_num):

	#line = "AIH 9BM9 Han 0 1 CG 0 CG"
	#line = "0 1DUI Poe 0 1 1RC 0 1RC "

	line_arr = line.split()

	ans = []

	bools = line_arr[0]

	dec = custom_base_to_decimal(bools, 32)

	#asa = number_to_base(dec, 2)
	asa = dec_to_binary(dec)

	asa = list(asa)

	asa = [bool(int(i)) for i in asa]

	ans = [str(i).upper() for i in asa]

	#print("liss",asa)

	date = line_arr[1]

	unix_timestamp = custom_base_to_decimal(date, 32)

	dt_object = datetime.datetime.fromtimestamp(unix_timestamp)

	# Format the datetime object as a string
	date = dt_object.strftime("%Y %B %d %H:%M:%S.%f")  # You can use any desired format here

	date = date[:-3]
	#print(date)
	ans.append(date)


	dealer = line_arr[2]
	#print(dealer)
	ans.append(dealer)

	dealer_tips = custom_base_to_decimal(line_arr[3], 32)
	#print(dealer_tips)
	ans.append(str(dealer_tips))

	bet_nums = custom_base_to_decimal(line_arr[4], 32)
	#print(bet_nums)
	ans.append(str(bet_nums))

	bets = custom_base_to_decimal(line_arr[5], 32)
	#print(bets)
	ans.append(str(bets))

	payout = custom_base_to_decimal(line_arr[6], 32)
	#print(payout)
	ans.append(str(payout))

	print(line_arr)
	profit = custom_base_to_decimal(line_arr[7], 32)
	#print(profit)
	ans.append(str(profit))

	ans.append(str(np.round((profit/bet_nums), decimals=2)))

	ans2 = str(line_num)+ ","
	for i in ans:
		ans2 += i + ","
	#print(ans2[:-1])

	return ans2[:-1]

#expand_line(2,1)


def collapse_line(line):
	
	#line = "1,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,1970 January 01 15:01:08.000,Poe,0,1,4300,0,4300,4300.0"
	line_arr = line.split(',')

	bool1 = ""

	#print(line_arr)
	for i in range(1,15):
		bool1 += str(0 + eval(line_arr[i].capitalize()))

	tBin = binary_to_dec(bool1)
	thBin = number_to_base(tBin, 32)

	#print(bool1, tBin, thBin)


	

	datetime_str = line_arr[15]

	datetime_obj = datetime.datetime.strptime(datetime_str, "%Y %B %d %H:%M:%S.%f")

	unix_timestamp = datetime_obj.timestamp()

	#print(unix_timestamp)

	#print(line_arr[16])

	#print(thBin, 
       #number_to_base(int(unix_timestamp), 32), 
	   #line_arr[16], 
	   #number_to_base(int(line_arr[17]), 32), 
	   #number_to_base(int(line_arr[18]), 32), 
	   #number_to_base(int(line_arr[19]), 32), 
	   #number_to_base(int(line_arr[20]), 32), 
	   #number_to_base(int(line_arr[21]), 32))
	asa = ""
	asa += str(thBin) + " "
	asa += str(number_to_base(int(unix_timestamp), 32)) + " "
	dealer = str(line_arr[16])
	dealers.add(dealer)
	asa += str(line_arr[16]) + " "
	asa += str(number_to_base(int(line_arr[17]), 32)) + " "
	asa += str(number_to_base(int(line_arr[18]), 32)) + " "
	asa += str(number_to_base(int(line_arr[19]), 32)) + " "
	asa += str(number_to_base(int(line_arr[20]), 32)) + " "
	asa += str(number_to_base(int(line_arr[21]), 32))
	
	return asa

def read_file_content(filename):
	f = open(filename, 'r', encoding = 'UTF-8')
	content = f.read()
	f.close()
	return content

################################################################################

In [ ]:
# Compression and decompression functions

def compress_data(raw):
    
	lines = raw.splitlines()
	lines = lines[1:1001]

	asa = ""
	print(lines[-4::])
	for i in lines:
		#print(i)
		asa += collapse_line(i) + "\n"

	#print(asa)

	return asa

def decompress_data(raw):
	lines = raw.splitlines()
	#lines = lines[:-1]

	asa = ""
	#print(lines[-4::])
	for line_num, line in enumerate(lines, start=1):
		#print(i)
		asa += expand_line(line, line_num) + "\n"

	#print(asa)

	return asa

def read_file_content(filename):
	f = open(filename, 'r', encoding = 'UTF-8')
	content = f.read()
	f.close()
	return content

#raw = read_file_content("raw_data/data_raw_0001.txt")
#compress_data(raw)
	

In [ ]:
# Analytics code from Q2

def get_stats(raw):

	
    
	lines = raw.splitlines()

	wins = []

	line = lines[1]

	lines = lines[2:1001]

	dealers:dict = {}


	npStats = np.zeros((1, 14), dtype=np.int32)
	npTips = np.zeros(1, dtype=np.int32)


	bettors = np.zeros(1000, dtype=np.int32)
	bets = np.zeros(1000, dtype=np.int32)
	payout = np.zeros(1000, dtype=np.int32)
	profit = np.zeros(1000, dtype=np.int32)

	#line = "1,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,1970 January 01 15:01:57.000,Homer,0,5,2500,0,2500,500.0"
	
	line_arr = line.split(',')


	#print(arr)

	# Color
	dealer = line_arr[16]
	tip = int(line_arr[17])

	check_dealer_one(dealer, dealers,npStats, npTips)
	#numbers
	get_dealer_stats_np(dealer, line_arr, dealers, npStats)
	
	get_tips_np(dealer, dealers, tip, npTips)

	bettors[0] = eval(line_arr[18])
	bets[0] = eval(line_arr[19])
	payout[0] = eval(line_arr[20])
	profit[0] = eval(line_arr[-2])

	

	# Read lines 2 to 101
	for line_number, line in enumerate(lines, 1):

		line_arr = line.split(',')


		#print(arr)

		# Color
		dealer = line_arr[16]
		tip = int(line_arr[17])

		asa, npStats, npTips = check_dealer(dealer, dealers, npStats, npTips)
		if not asa:
			a = 1
		#numbers
		get_dealer_stats_np(dealer, line_arr, dealers, npStats)
		
		#tips
		get_tips_np(dealer, dealers, tip, npTips)

		bettors[line_number] = eval(line_arr[18])
		bets[line_number] = eval(line_arr[19])
		payout[line_number] = eval(line_arr[20])
		profit[line_number] = eval(line_arr[-2])


	#print(dealers)

	asa = get_number_analytics(npStats, dealers)
	
	asa += "\n\n\n" + get_tips_analytics(npTips, dealers)

	
	asa += "\n\n\n" +get_average_analytics(bettors, bets, payout, profit)

	#print(asa)

	asa += "\n\n##### END #####"

	return asa



	#print(npStats)
	#print(npTips)





def get_average_analytics(bettors, bets, payout, profit):
	asa = "Average Analytics\n"
	asa += "\nAverage Number of Bettors," + str(np.round(np.mean(bettors), decimals=2))
	asa += "\nAverage Bet Size," + str(np.round(np.mean(bets), decimals=2))
	asa += "\nAverage Bet Size per Bettor," + str(np.round(sum(bets)/sum(bettors), decimals=2))
	asa += "\nAverage Payout," + str(np.round(np.mean(payout), decimals=2))
	asa += "\nAverage Payout per Bettor," + str(np.round(sum(payout)/sum(bettors), decimals=2))
	asa += "\nAverage Profit per Spin," + str(np.round(np.mean(profit), decimals=2))
	return asa

def get_number_analytics(npStats, dealers):
	asa ="Number Analytics\n"

	asa += "\n" + total_number_analytics(npStats=npStats)

	sorted_dealers = dict(sorted(dealers.items()))
	for key, value in sorted_dealers.items():
		asa += "\n" + dealer_number_analytics(key, value, npStats)

	return asa


def total_number_analytics(npStats):

	stats = np.sum(npStats, axis=0)
	asa = "Amount,"
	for i in range(len(stats)):
		asa += str(stats[i]) + ","
	asa = asa[:-1]

	pr100 = calculate_percentages(stats, 1000)
	asa += "\nPercentage,"
	for i in range(len(pr100)):
		asa += str(np.round(pr100[i], decimals=1)) + ","

	return asa[:-1]

def dealer_number_analytics(dealer, dealerIndex, npStats):

	asa = "Amount (" + dealer + ")," 
	for i in range(len(npStats[dealerIndex])):
		asa += str(npStats[dealerIndex, i]) + ","
	asa = asa[:-1]

	pr100 = calculate_percentages(npStats[dealerIndex], 1000)
	asa += "\nPercentage (" + dealer + ")," 
	for i in range(len(pr100)):
		asa += str(np.round(pr100[i], decimals=1)) + ","

	return asa[:-1]

def get_tips_analytics(npTips, dealers):
	asa = "Tip Analytics\n"

	asa += "\n" + total_tips_analytics(npTips)

	sorted_dealers = dict(sorted(dealers.items()))
	for key, value in sorted_dealers.items():
		asa += "\n" + dealer_tips_analytics(key, value, npTips)
	
	return asa

def total_tips_analytics(npTips):
	asa = "Tips (Total)," + str(np.sum(npTips))
	return asa

def dealer_tips_analytics(dealer, dealerIndex, npTips):
	asa = "Tips (" + dealer + ")," + str(npTips[dealerIndex])
	return asa

def check_dealer(dealer: str, dealers:dict, npStats:np.ndarray[np.int32], npTips:np.ndarray[np.int32]): # type: ignore
	if dealer not in dealers:
		dealers[dealer] = len(dealers)
		npStatsNew = np.vstack((npStats, np.zeros(14, dtype=np.int32)), dtype=np.int32)
		#print(npStats.size)
		npTipsNew = np.append(npTips, 0)
		return False , npStatsNew, npTipsNew
	else:
		return True , npStats, npTips
	
def check_dealer_one(dealer: str, dealers:dict, npStats:np.ndarray[np.int32], npTips:np.ndarray[np.int32]): # type: ignore
	if dealer not in dealers:
		dealers[dealer] = 0
		return False
	else:
		return True
	
def get_dealer_stats(dealer:str, line_arr:list, dealers:dict, stats:list):
	index = dealers[dealer]
	for i in range(14):
		stats[index][i] += eval(line_arr[i+1].capitalize())

def get_dealer_stats_np(dealer:str, line_arr:list, dealers:dict, npStats:np.ndarray[np.int32]): # type: ignore
	index = dealers[dealer]
	#print(dealers)
	for i in range(14):
		npStats[index, i] += eval(line_arr[i+1].capitalize())

def get_tips(dealer, dealers, tip, tips):
	index = dealers[dealer]
	tips[index] += tip

def get_tips_np(dealer, dealers, tip, npTips):
	index = dealers[dealer]
	npTips[index] += tip

def calculate_percentages(data, size):
    # Calculate the total sum of the dat
    
    # Calculate the percentages using element-wise division and multiplication
    percentages = np.true_divide(data, size) * 100
    
    return percentages

In [ ]:
### compressing algorithm ###
def compress(raw):
    ### add logic here
    compressed = raw
    compressed = compress_data(raw)
    ###
    return compressed
#############################

In [ ]:
### decompressing algorithm ###
def decompress(compressed):
    ### add logic here
    decompressed = compressed
    header = "Spin Number,Red,Black,Even,Odd,1 to 18,19 to 36,1st 12,2nd 12,3rd 12,1st Column,2nd Column,3rd Column,Prime,Divisible by 4,Timestamp,Dealer,Dealer Tips,Number of Bettors,Bets,Payouts (incl. Bet),Profit,Profit per Bettor\n"
    footer = "\n\n"
    decompressed = header + decompress_data(compressed) + footer

    stats = get_stats(decompressed)
    
    decompressed += stats
    ###
    return decompressed
###############################

In [ ]:
####### !!! DO NOT MODIFY !!! #######
def read_file_content(filename):
    f = open(filename, 'r', encoding = 'UTF-8')
    content = f.read()
    f.close()
    return content

def write_file_content(filename, content):
    f = open(filename, 'w', encoding = 'UTF-8')
    f.write(content)
    f.close()
    return None

def do_compression(content_raw, write_file_name):
    content_compressed = compress(content_raw)
    write_file_content(write_file_name, content_compressed)
    return content_compressed

def do_decompression(content_compressed, write_file_name):
    content_decompressed = decompress(content_compressed)
    write_file_content(write_file_name, content_decompressed)
    return content_decompressed

def compression_decompression(file_number = '0001'):
    base_filepath = '_data/data_'
    
    ### read file raw content from file with specified name
    content_raw = read_file_content('raw' + base_filepath + 'raw_'  + file_number + '.txt')
    
    ### compress raw data and then decompress compressed data content. Save results in txt files.
    content_compressed = do_compression(content_raw, 'compressed' + base_filepath + 'compressed_' + file_number + '.txt')
    content_decompressed = do_decompression(content_compressed, 'decompressed' + base_filepath + 'decompressed_' + file_number + '.txt')
    
    ### encode content to UTF-8 from string type to remove native string class overhead
    content_raw_encoded = content_raw.encode() ### UTF-8
    content_compressed_encoded = content_compressed.encode() ### UTF-8
    content_decompressed_encoded = content_decompressed.encode() ### UTF-8
    
    return content_raw, content_compressed, content_decompressed, content_raw_encoded, content_compressed_encoded, content_decompressed_encoded

def evaluate_performance(file_number = '0001'):
    raw, compressed, decompressed, raw_encoded, compressed_encoded, decompressed_encoded = compression_decompression(file_number)
    
    ### evaluate size - each character in a UTF-8 encoded string represents 1 byte
    raw_size = len(raw_encoded) # bytes
    compressed_size = len(compressed_encoded) # bytes
    decompressed_size = len(decompressed_encoded) # bytes
    
    # ### print size and type
    # print(raw_size, type(raw_encoded))
    # print(compressed_size, type(compressed_encoded))
    # print(decompressed_size, type(decompressed_encoded))
    
    ### calculate metrics, and check that output matches input perfectly
    compression_ratio = raw_size/compressed_size
    
    decompressed_hash = hashlib.md5(decompressed_encoded).hexdigest()
    raw_hash = hashlib.md5(raw_encoded).hexdigest()
    
    lossless = decompressed_hash == raw_hash ## md5 checksum
    
    return compression_ratio, lossless
####### !!! DO NOT MODIFY !!! #######

### PERFORMANCE
REQUIREMENT: Generated output file should be identical to the raw file. (i.e. should be lossless)

OBJECTIVE: Maximise compresssion ratio.

In [ ]:
### replace file_number with the number of the txt file you want to test on
file_number = '0001'

### run cell to evaluate compression performance
compression_ratio, lossless = evaluate_performance(file_number)

performance = 'Compression Ratio: ' + str(compression_ratio) + '\nLossless: ' + str(lossless)
print(performance) 